In [3]:
%config IPCompleter.greedy=True
from IPython.display import IFrame

import pandas as pd
import numpy as np
# import string
import math

# import scipy.stats as sts

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn import preprocessing as prep
# import sklearn.metrics as metrics
import sklearn.model_selection as model_selection
# from sklearn import discriminant_analysis as disan
# from sklearn import calibration as calib
# from sklearn import linear_model as lm
# from sklearn import svm
# from sklearn import gaussian_process as gaup
# from sklearn import mixture as mix
# from sklearn import tree
# from sklearn import ensemble as ens

import tensorflow as tf
from tensorflow import keras

# from keras import models as kermdls
# from keras import layers as kerlrs
# from keras import metrics as kmetrics

# from hyperas import optim
# from hyperas.distributions import choice, uniform
# from hyperopt import Trials, STATUS_OK, tpe

# import pickle

# import nilearn as nl
# from nilearn import plotting, image
# from nilearn import datasets
# import nibabel as nb
import h5py

import os

import time

In [4]:
TEST_IDS = [map_id.split('.')[0] for map_id in sorted(os.listdir('00_Data/fMRI_test'))]
TRAIN_IDS = [map_id.split('.')[0] for map_id in sorted(os.listdir('00_Data/fMRI_train'))]

In [5]:
data = pd.read_csv('00_Data/train_scores_full.csv')
data

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641
3,10007,38.617381,49.197021,65.674285,40.151376,34.096421
4,10008,35.326582,15.769168,65.782269,44.643805,50.448485
...,...,...,...,...,...,...
5872,21654,53.103634,50.951656,62.168022,49.389400,53.020847
5873,21665,38.246437,48.018227,59.522285,45.697098,53.208160
5874,21674,69.414169,58.593918,60.298779,49.865669,47.863167
5875,21693,62.009209,54.272484,60.474388,52.325031,52.989803


In [6]:
data.isnull().sum()

Id              0
age             0
domain1_var1    0
domain1_var2    0
domain2_var1    0
domain2_var2    0
dtype: int64

In [7]:
print('Dataset length: ', len(data.index))

Dataset length:  5877


In [8]:
def inputs_gen(idxs, labels):
    for idx, labs in zip(idxs, labels):
        # FNC inputs
        df_fnc = pd.read_csv('00_Data/fnc_csv/{0}.csv'.format(idx), index_col=0)
        X_fnc = np.array(df_fnc.values).reshape(-1)

        # Loading inputs
        df_loading = pd.read_csv('00_Data/loading_csv/{0}.csv'.format(idx), index_col=0)
        X_loading = np.array(df_loading.values).reshape(-1)

        #MRI inputs
        patient_SM = h5py.File('00_Data/fMRI_train/{0}.mat'.format(idx), mode='r')
        patient_SM = np.array(patient_SM.get('SM_feature'))

        k = 2
        ki_padding = 3

        arr_regions = []
        for i in range(patient_SM.shape[0]):
            sample_map = patient_SM[i,:,:,:]
            # padding MRI map
            map_shape = sample_map.shape
            shape_pad = ((map_shape[0]//k + 1)*k - map_shape[0],
                         (map_shape[1]//k + 1)*k - map_shape[1],
                         (map_shape[2]//k + 1)*k - map_shape[2])

            npad = ((shape_pad[0]//2, (shape_pad[0]//2 if shape_pad[0]%2==0 else shape_pad[0]//2+1)),    
                    (shape_pad[1]//2, (shape_pad[1]//2 if shape_pad[1]%2==0 else shape_pad[1]//2+1)),    
                    (shape_pad[2]//2, (shape_pad[2]//2 if shape_pad[2]%2==0 else shape_pad[2]//2+1)))

            sample_map_padded = np.pad(sample_map, pad_width=npad, mode='constant', constant_values=0)

            sx = sample_map_padded.shape[0] / k
            sy = sample_map_padded.shape[1] / k
            sz = sample_map_padded.shape[2] / k
            for kz in range(k):
                for ky in range(k):
                    for kx in range(k):
                        ki_region = sample_map_padded[int(kx*sx): int(kx*sx + sx - 1), 
                                                     int(ky*sy): int(ky*sy + sy - 1), 
                                                     int(kz*sz): int(kz*sz + sz - 1)]
                        # padding i-th region by 3 pixels
                        ki_region_padded = np.pad(ki_region, pad_width=ki_padding, mode='constant', constant_values=0)
                        arr_regions.append(ki_region_padded)
        X_mri = np.stack(arr_regions, axis=3)

    #     X = (X_mri, X_fnc, X_loading)
        yield X_mri, X_fnc, X_loading, labs

In [9]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [10]:
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

In [11]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [12]:
def _dtype_feature(ndarray):
        """match appropriate tf.train.Feature class with dtype of ndarray. """
        assert isinstance(ndarray, np.ndarray)
        dtype_ = ndarray.dtype
        if dtype_ == np.float64 or dtype_ == np.float32:
            return lambda array: tf.train.Feature(float_list=tf.train.FloatList(value=array))
        elif dtype_ == np.int64:
            return lambda array: tf.train.Feature(int64_list=tf.train.Int64List(value=array))
        else:  
            raise ValueError("The input should be numpy ndarray. \
                               Instaed got {}".format(ndarray.dtype))

In [12]:
def serialize_fullproto_pair(x1, x2, x3, y):
#     feat_x1 = _bytes_feature(x1.tostring())
#     feat_x2 = _bytes_feature(x2.tostring())
#     feat_x3 = _bytes_feature(x3.tostring())
#     feat_y = _bytes_feature(y.tostring())
 
#     proto_x1 = tf.train.FeatureList(feature=[feat_x1])
#     proto_x2 = tf.train.FeatureList(feature=[feat_x1, feat_x2, feat_x3])
#     proto_x3 = tf.train.FeatureList(feature=[feat_x1, feat_x2, feat_x3])
    
#     proto_y = feat_y
#     print(x1.shape)
#     print(x2.shape)
#     print(x3.shape)
#     feats2 = []
#     for i in x2:
#         feats2.append(tf.train.Feature(float_list=tf.train.FloatList(value=[i])))
        
        
    y_feats = tf.train.Features(feature={'age': tf.train.Feature(float_list=tf.train.FloatList(value=[y[0]])),
                                         'domain1_var1': tf.train.Feature(float_list=tf.train.FloatList(value=[y[1]])),
                                         'domain1_var2': tf.train.Feature(float_list=tf.train.FloatList(value=[y[2]])),
                                         'domain2_var1': tf.train.Feature(float_list=tf.train.FloatList(value=[y[3]])),
                                         'domain2_var2': tf.train.Feature(float_list=tf.train.FloatList(value=[y[4]]))})    
        
    x1 = x1.flatten()  
    print(x1.shape)
    proto_x1 = tf.train.FeatureList(feature=[tf.train.Feature(float_list=tf.train.FloatList(value=[value])) for value in x1])
    
#     proto_x1d1 = tf.train.FeatureList(feature=[tf.train.Feature(float_list=tf.train.FloatList(value=[value])) for value in x1[0]])
#     proto_x1d2 = tf.train.FeatureList(feature=[tf.train.Feature(float_list=tf.train.FloatList(value=[value])) for value in x1[1]])
#     proto_x1d3 = tf.train.FeatureList(feature=[tf.train.Feature(float_list=tf.train.FloatList(value=[value])) for value in x1[2]])
#     proto_x1d4 = tf.train.FeatureList(feature=[tf.train.Feature(float_list=tf.train.FloatList(value=[value])) for value in x1[3]])
    
    proto_x2 = tf.train.FeatureList(feature=[tf.train.Feature(float_list=tf.train.FloatList(value=[value])) for value in x2])
#     proto_x2 = tf.train.FeatureList(feature=feats2)
    proto_x3 = tf.train.FeatureList(feature=[tf.train.Feature(float_list=tf.train.FloatList(value=[value])) for value in x3])
    
    proto_y = tf.train.FeatureList(feature=[tf.train.Feature(float_list=tf.train.FloatList(value=[value])) for value in y])

    sample_protobuf = tf.train.SequenceExample(context=y_feats,
                                               feature_lists=tf.train.FeatureLists(feature_list={'x1': proto_x1, 'x2': proto_x2, 'x3': proto_x3}))
    return sample_protobuf.SerializeToString()

In [13]:
def serialize_fullproto(x1, x2, x3, y):
    feat_x1 = _bytes_feature(x1.tostring())
    feat_x2 = _bytes_feature(x2.tostring())
    feat_x3 = _bytes_feature(x3.tostring())
    feat_y = _bytes_feature(y.tostring())
 
    #proto_x = tf.train.FeatureList(feature=[feat_x1, feat_x2, feat_x3])
    
#     proto_y = feat_y

    sample_protobuf = tf.train.Example(features=tf.train.Features(feature={'x1': feat_x1, 'x2': feat_x2, 'x3': feat_x3, 'y': feat_y}))
    return sample_protobuf.SerializeToString()

In [14]:
train, test = model_selection.train_test_split(data, test_size=0.2, shuffle=True, random_state=30)
train, val = model_selection.train_test_split(train, test_size=0.2, shuffle=True, random_state=30)
print(len(train.index))

3760


In [15]:
df = train.head()

In [19]:
df

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
563,11216,47.470203,42.356793,45.506630,61.463918,54.559339
206,10429,55.456978,36.410912,64.142701,36.430656,47.880362
5464,10841,40.247440,49.099689,63.119778,46.581844,53.951828
283,10590,64.203107,66.665076,25.882906,61.022117,62.865055
5810,19892,44.378983,48.842289,60.420139,45.340927,52.699509


In [16]:
type(train)

pandas.core.frame.DataFrame

In [17]:
tfr_train = 'train_1.tfrecord'
tfr_val = 'val.tfrecord'
tfr_test = 'test.tfrecord'

ops = tf.io.TFRecordOptions(compression_type="GZIP")

In [18]:
with tf.io.TFRecordWriter(tfr_train, options=ops) as protowriter:
    for i, (x1, x2, x3, y) in enumerate(inputs_gen(df['Id'].values, df[['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']].values)):
        sample = serialize_fullproto(x1, x2, x3, y)
        protowriter.write(sample)
        print("Wrote sample #{}".format(i + 1))
    protowriter.flush()

Wrote sample #1
Wrote sample #2
Wrote sample #3
Wrote sample #4
Wrote sample #5
